In [28]:
import requests
import pandas as pd
import os 
import numpy as np

In [94]:
features = ['avg-block-size',
'avg-confirmation-time',
'blocks-size',
'cost-per-transaction',
'cost-per-transaction-percent',
'difficulty',
'estimated-transaction-volume',
'estimated-transaction-volume-usd',
'hash-rate',
'market-cap',
'market-price',
'median-confirmation-time',
'mempool-count',
'mempool-growth',
'mempool-size',
'miners-revenue',
'n-transactions',
'n-transactions-excluding-popular',
'n-transactions-per-block',
'n-transactions-total',
'n-unique-addresses',
'output-volume',
'total-bitcoins',
'trade-volume',
'transaction-fees',
'transaction-fees-usd',
'transactions-per-second',
]

In [4]:
for feature in features:
    url = f"https://api.blockchain.info/charts/{feature}"
    params = {
        "timespan": "10years",
        # "rollingAverage": "8hours",
        "format": "json",
        'sampled':'false'
    }

    response = requests.get(url, params=params)
    data = response.json()

    try:
        df = pd.DataFrame([])
        for i, row in enumerate(data['values']):
            date = pd.to_datetime(row['x'], unit='s')
            price = row['y']
            df = pd.concat([df, pd.DataFrame([{'date':date, 'price':price}])], ignore_index=True)
            # print(date, price)
        df.to_csv(f"data_new/{feature}.csv", index=False)
    except:
        print(f'Error in {feature}')

Error in pools


In [89]:
files = os.listdir('data_new')
df = pd.read_csv(f'data_new/{files[0]}')
df.columns = ['date', f'{files[0][:-4]}']
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df

,avg-block-size
date,
2014-04-15,0.241621
2014-04-16,0.271066
2014-04-17,0.223713
2014-04-18,0.205560
2014-04-19,0.193319
...,...
2024-04-07,1.884625
2024-04-08,1.937860
2024-04-09,1.803056


In [90]:
x1 = pd.read_csv('data_new/market-cap.csv')
x1['date'] = pd.to_datetime(x1['date'])
x1.set_index('date', inplace=True)
x1 = x1.resample('D').mean()
x1

,price
date,
2014-04-15,5.814155e+09
2014-04-16,6.617884e+09
2014-04-17,6.661624e+09
2014-04-18,6.308121e+09
2014-04-19,6.106323e+09
...,...
2024-04-07,1.364788e+12
2024-04-08,1.399943e+12
2024-04-09,1.380335e+12


In [91]:
x2 = pd.read_csv('data_new/total-bitcoins.csv')
x2['date'] = pd.to_datetime(x2['date'])
x2.set_index('date', inplace=True)
x2 = x2.resample('D').mean()
x2

,price
date,
2014-04-15,1.264937e+07
2014-04-16,1.265370e+07
2014-04-17,1.265771e+07
2014-04-18,1.266182e+07
2014-04-19,1.266583e+07
...,...
2024-04-07,1.967582e+07
2024-04-08,1.967685e+07
2024-04-09,1.967780e+07


In [92]:
for i in np.arange(1, len(files)):
    df_i = pd.read_csv(f'data_new/{files[i]}')
    df_i.columns = ['date', f'{files[i][:-4]}']
    df_i['date'] = pd.to_datetime(df_i['date'])
    df_i.set_index('date', inplace=True)
    try:
        df = df.join(df_i)
    except: # for data which is not on a daily basis
        df_i = df_i.resample('D').mean()
        df = pd.merge(df, df_i, left_index=True, right_index=True)
df['market-cap'] = np.array(x1['price']) # replace Nan cols for two specific data
df['total-bitcoins'] = np.array(x2['price'])
df.to_csv('BTC_new.csv',index=True)
df

,avg-block-size,avg-confirmation-time,blocks-size,cost-per-transaction-percent,cost-per-transaction,difficulty,estimated-transaction-volume-usd,estimated-transaction-volume,hash-rate,market-cap,...,n-transactions-per-block,n-transactions-total,n-transactions,n-unique-addresses,output-volume,total-bitcoins,trade-volume,transaction-fees-usd,transaction-fees,transactions-per-second
date,,,,,,,,,,,,,,,,,,,,,
2014-04-15,0.241621,NaN,17700.145907,4.312103,31.461496,6.119726e+09,6.737795e+07,146588.522890,5.932158e+04,5.814155e+09,...,400.049451,36853958.0,72809.0,156359.0,669588.742590,1.264937e+07,1.547115e+07,9.674245e+03,16.499545,NaN
2014-04-16,0.271066,NaN,17744.199359,2.176376,33.637923,6.119726e+09,1.235655e+08,236262.993009,4.532777e+04,6.617884e+09,...,431.408805,36926901.0,68594.0,149841.0,808972.686487,1.265370e+07,1.702180e+07,8.876057e+03,14.274446,NaN
2014-04-17,0.223713,NaN,17787.130813,3.960223,36.243881,6.978843e+09,6.833143e+07,129836.087704,6.036411e+04,6.661624e+09,...,385.739645,36995367.0,65190.0,131868.0,814063.297992,1.265771e+07,1.618354e+07,8.840666e+03,14.995538,NaN
2014-04-18,0.205560,NaN,17824.879464,6.280938,40.416473,6.978843e+09,4.413939e+07,88597.737449,5.411954e+04,6.308121e+09,...,356.225000,37060438.0,56996.0,128648.0,781666.572318,1.266182e+07,1.000750e+07,7.357991e+03,12.160113,NaN
2014-04-19,0.193319,NaN,17857.786874,7.890680,39.914189,6.978843e+09,3.141112e+07,65153.437509,5.654798e+04,6.106323e+09,...,336.572327,37117369.0,53515.0,118692.0,460095.566680,1.266583e+07,5.372537e+06,7.737646e+03,12.613149,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-07,1.884625,41.076088,562982.133215,1.500297,142.915261,8.312700e+13,5.011808e+09,72211.397350,7.024851e+08,1.364788e+12,...,3094.882353,985002826.0,526130.0,652726.0,554943.049164,1.967582e+07,1.684330e+08,1.506788e+06,21.723003,2.850000
2024-04-08,1.937860,44.144762,563302.582414,0.635671,159.310532,8.312700e+13,1.141858e+10,159660.840237,6.528979e+08,1.399943e+12,...,2883.651899,985528264.0,455617.0,685180.0,982666.187226,1.967685e+07,1.995999e+08,2.366374e+06,33.119632,3.633333
2024-04-09,1.803056,71.778546,563608.974102,0.615871,148.461847,8.312700e+13,1.093957e+10,156080.237383,6.033108e+08,1.380335e+12,...,3108.294521,985984696.0,453811.0,683771.0,837041.011677,1.967780e+07,4.625718e+08,3.306938e+06,47.245769,2.900000
